In [28]:
import random as rng
from itertools import product

In [29]:
class Card:
    def __init__(self,fig,col):
        self.colour = col
        self.figure = fig
        self.slownik =[
            {
            '2' : '2',
            '3' : '3',
            '4' : '4',
            '5' : '5',
            '6' : '6',
            '7' : '7',
            '8' : '8',
            '9' : '9',
            '10' : '10',
            'J' : 'Jack',
            'Q' : 'Queen',
            'K' : 'King',
            'A' : 'Ace'},
            {
            'H': 'Hearts',
            'D': 'Diamonds',
            'C': 'Clubs',
            'S': 'Spades'}
            ]
    
        rank_dict = {figure : rank for rank,figure in enumerate(self.slownik[0].keys())}
        self.rank_of_card = rank_dict[self.figure]
    def __str__(self):
        return f'{self.slownik[0][self.figure]} of {self.slownik[1][self.colour]}'
    
    def __repr__(self):
        return f'{self.figure}{self.colour}'

    def __lt__(self,other):
        return self.rank_of_card < other.rank_of_card
    def __le__(self,other):
        return self.rank_of_card <= other.rank_of_card
    def __gt__(self,other):
        return self.rank_of_card > other.rank_of_card
    def __ge__(self,other):
        return self.rank_of_card >= other.rank_of_card
    def __eq__(self,other):
        return self.rank_of_card == other.rank_of_card

c = Card('A', 'D')
print(c)


Ace of Diamonds


definicja obiektu <u>KARTA</u>, z atrybutamni

In [37]:
class Deck:
    def __init__(self):
        figures = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
        colours = 'H D C S'.split()
        self.talia = []
        for i in product (figures, colours):
            self.talia.append(Card(i[0], i[1]))
    def shuffle(self):
        temp_deck = self.talia[:]
        self.talia = []
        while temp_deck:
            self.talia.append(temp_deck.pop(rng.randint(0,len(temp_deck)-1)))
        #print(temp_deck)
    def draw(self, n = 1):
        card_list = []
        while n:
            card_list.append(self.talia.pop())
            n-=1
        return card_list
    def reset(self):
        self.__init__()
        self.shuffle()
t = Deck()
t.shuffle()


In [31]:
class Hand:
    def __init__(self):
        self.cards_in_hand = []
        self.cards_on_table= []
        self.high_card = None
    def get_card(self, talia: Deck, n :int):
        for card in talia.draw(n):
            self.cards_in_hand.append(card)
    def find_high_card(self):
        self.high_card = (self.cards_in_hand[0])
        for card in self.cards_in_hand+self.cards_on_table:
            if card.rank_of_card > self.high_card.rank_of_card:
                self.high_card = card

    def all_cards(self):
        return sorted(self.cards_in_hand + self.cards_on_table)
    def reset_hand(self):
        self.cards_in_hand = []
        self.cards_on_table= []
        self.high_card = None
h1 = Hand()
h1.get_card(t,2)
print(h1.cards_in_hand)

[9D, 9C]


In [46]:
class Player:
    def __init__(self, name = ''):        
        self.name = name
        self.balance = 0
        self.hand = Hand()
        self.hand_history = []
    def __repr__(self):
        return f"""
player name:{self.name} 
current $$ :{self.balance} 
cards in hand: {self.hand.cards_in_hand}cards on the table {self.hand.cards_on_table}: 
high card :{self.hand.all_cards()[-1]}
all cards (ordered){self.hand.all_cards()}
"""
p1 = Player ('Ziutek')
p1.hand.get_card(t,2)
p2 = Player ('Fiutek')
p2.hand.get_card(t,2)
print(p1)
print(p2)



player name:Ziutek 
current $$ :0 
cards in hand: [3H, JD]cards on the table []: 
high card :Jack of Diamonds
all cards (ordered)[3H, JD]


player name:Fiutek 
current $$ :0 
cards in hand: [QS, 2S]cards on the table []: 
high card :Queen of Spades
all cards (ordered)[2S, QS]



In [47]:
class Game:
    def __init__(self, players, blind = 0):
        self.start_time = None  # to work on
        self.end_time   = None  #  as above
        self.list_of_players= [] 
        self.deck = Deck()
        self.deck.shuffle()

        for player in players:
            self.list_of_players.append(player)
        
        self.loot = blind
        self.blind_value = 0
        self.small_blind_value = self.blind_value//2
        self.loot = self.blind_value + self.small_blind_value

    def turn_flop(self):
        flop = self.deck.draw(3)
        for player in self.list_of_players:
            for card in flop:
                player.hand.cards_on_table.append(card)
        for player in self.list_of_players:
            player.hand.all_cards()[-1]
        print('*********FLOP********')

    def turn_turn(self):
        turn = self.deck.draw(1)
        for player in self.list_of_players:
            player.hand.cards_on_table.append(turn[0])
        for player in self.list_of_players:
            player.hand.all_cards()[-1]
        print('*********TURN********')
    def turn_river(self):
        turn = self.deck.draw(1)
        for player in self.list_of_players:
            player.hand.cards_on_table.append(turn[0])
        for player in self.list_of_players:
            player.hand.all_cards()[-1]
        print('*********RIVER********')

    def end_game(self):
        self.deck.reset()
        for player in self.list_of_players:
            player.hand.reset_hand()
g1 = Game([p1,p2], blind = 100)
g1.turn_flop()
print(p1)
print(p2)
g1.turn_turn()
print(p1)
print(p2)
g1.turn_river()
print(p1)
print(p2)
g1.end_game()

*********FLOP********

player name:Ziutek 
current $$ :0 
cards in hand: [3H, JD]cards on the table [AC, 3H, 4S]: 
high card :Ace of Clubs
all cards (ordered)[3H, 3H, 4S, JD, AC]


player name:Fiutek 
current $$ :0 
cards in hand: [QS, 2S]cards on the table [AC, 3H, 4S]: 
high card :Ace of Clubs
all cards (ordered)[2S, 3H, 4S, QS, AC]

*********TURN********

player name:Ziutek 
current $$ :0 
cards in hand: [3H, JD]cards on the table [AC, 3H, 4S, KD]: 
high card :Ace of Clubs
all cards (ordered)[3H, 3H, 4S, JD, KD, AC]


player name:Fiutek 
current $$ :0 
cards in hand: [QS, 2S]cards on the table [AC, 3H, 4S, KD]: 
high card :Ace of Clubs
all cards (ordered)[2S, 3H, 4S, QS, KD, AC]

*********RIVER********

player name:Ziutek 
current $$ :0 
cards in hand: [3H, JD]cards on the table [AC, 3H, 4S, KD, 5D]: 
high card :Ace of Clubs
all cards (ordered)[3H, 3H, 4S, 5D, JD, KD, AC]


player name:Fiutek 
current $$ :0 
cards in hand: [QS, 2S]cards on the table [AC, 3H, 4S, KD, 5D]: 
high card 